#### Setup

In [17]:
from experiments.main_functions.dataloader import dataloader
from experiments.main_functions.plots import hartley_fourier
from experiments.main_functions.slicing import dataslicing
from experiments.main_functions.DHT import dataDHT
from experiments.main_functions.DFT import dataDFT
from experiments.main_functions.pooling import datapooling
from experiments.main_functions.utils import NormalizeData
import numpy as np

import plotly.graph_objects as go
from plotly.subplots import make_subplots

#### Loading data

In [19]:
datapath = ("/home/natalia/Documentos/Dados/Tsinghua BCI Lab - Benchmark Dataset/")

In [22]:
suj = 1
elec = 61
tr = 1
seconds_off = 1


signals15, labels15 = dataloader(subject=suj, electrode=elec, stimulus_frequency=[15], trial=tr, sec_off=seconds_off, path=datapath)
signals10, labels10 = dataloader(subject=suj, electrode=elec, stimulus_frequency=[10], trial=tr, sec_off=seconds_off, path=datapath)
signals8, labels8 = dataloader(subject=suj, electrode=elec, stimulus_frequency=[8], trial=tr, sec_off=seconds_off, path=datapath)
signals12, labels12 = dataloader(subject=suj, electrode=elec, stimulus_frequency=[12], trial=tr, sec_off=seconds_off, path=datapath)

print(signals15.shape, signals10.shape, signals8.shape, signals12.shape)

TypeError: dataloader() got an unexpected keyword argument 'sec_off'

#### Plotting the signal and its Fourier and Hartley Transforms

In [7]:
s = [signals8, signals10, signals12, signals15]
sf = [8, 10, 12, 15]

for idx, signal in enumerate(s):
    hartley_fourier(signal=np.reshape(signal, (1000)), stimulus_frequency=sf[idx], sampling_frequency=250)

#### Slicing the signal

In [8]:
# adding an extra dimension
signals15_ = np.reshape(signals15, (1, signals15.shape[0], 1))
print('Forma do sinal de entrada:', signals15_.shape)
signals15_blks = dataslicing(data=signals15_, levels=3)
print('Número de blocos após fatiamento:', len(signals15_blks))

for blk in signals15_blks:
    print('Tamanho do bloco:', blk.shape)

Forma do sinal de entrada: (1, 1000, 1)
nivel: 0
nivel: 1
nivel: 2
Número de blocos após fatiamento: 21
Tamanho do bloco: (1, 1000, 1)
Tamanho do bloco: (1, 250, 1)
Tamanho do bloco: (1, 250, 1)
Tamanho do bloco: (1, 250, 1)
Tamanho do bloco: (1, 250, 1)
Tamanho do bloco: (1, 62, 1)
Tamanho do bloco: (1, 62, 1)
Tamanho do bloco: (1, 62, 1)
Tamanho do bloco: (1, 62, 1)
Tamanho do bloco: (1, 62, 1)
Tamanho do bloco: (1, 62, 1)
Tamanho do bloco: (1, 62, 1)
Tamanho do bloco: (1, 62, 1)
Tamanho do bloco: (1, 62, 1)
Tamanho do bloco: (1, 62, 1)
Tamanho do bloco: (1, 62, 1)
Tamanho do bloco: (1, 62, 1)
Tamanho do bloco: (1, 62, 1)
Tamanho do bloco: (1, 62, 1)
Tamanho do bloco: (1, 62, 1)
Tamanho do bloco: (1, 62, 1)


In [11]:
concat = np.concatenate(signals15_blks, axis=1)

In [14]:
data = [concat]

# Define the colors for each level
colors = [['skyblue', 'steelblue', 'navy']]  

# Create a figure with subplots
fig = make_subplots(rows=len(data), cols=1, shared_xaxes=True, vertical_spacing=0.05,
                    subplot_titles=("Input Signal"))

# Add traces for each dataset
for i in range(len(data)):
    # Slice the data into three levels for each dataset
    level1 = data[i][:, :1000, :]
    level2 = data[i][:, 1000:1000+(4*250), :]
    level3 = data[i][:, 1000+(4*250):, :]

    # Create traces for each level for the current dataset
    trace1 = go.Scatter(x=np.arange(len(level1[0])), y=level1[0,:,0], mode='lines', name='Level 1', line=dict(color=colors[i][0], width=2))
    trace2 = go.Scatter(x=np.arange(len(level1[0]), len(level1[0])+len(level2[0])), y=level2[0,:,0], mode='lines', name='Level 2', line=dict(color=colors[i][1], width=2))
    trace3 = go.Scatter(x=np.arange(len(level1[0])+len(level2[0]), len(data[i][0])), y=level3[0,:,0], mode='lines', name='Level 3', line=dict(color=colors[i][2], width=2))

    # Add traces to the subplot
    fig.add_trace(trace1, row=i+1, col=1)
    fig.add_trace(trace2, row=i+1, col=1)
    fig.add_trace(trace3, row=i+1, col=1)

# Update layout
#fig.update_layout( title_text="Data Comparison", showlegend=True)

# Show the figure
# Update the layout and display the figure
fig.update_layout(height=400*len(data), title='Slicing', template='plotly_white')
fig.show()

#### Applying the DHT and pooling

In [41]:
grouped = []

for block in range(len(signals15_blks)):
    dhtdata = dataDHT(signals15_blks[block])
    print('Tamanho após transformada:', dhtdata.shape)
    datapool = datapooling(dhtdata, axis=1, width=1)
    print('Tamanho após agrupamento:', datapool.shape)
    grouped.append(datapool)
groupeddata = jnp.concatenate(grouped, axis=1)
print('Tamanho do sinal com blocos concatenados:', groupeddata.shape)
norm_groupeddata = NormalizeData(groupeddata)

Tamanho após transformada: (1, 500, 1)
Tamanho após agrupamento: (1, 500, 1)
Tamanho após transformada: (1, 125, 1)
Tamanho após agrupamento: (1, 125, 1)
Tamanho após transformada: (1, 125, 1)
Tamanho após agrupamento: (1, 125, 1)
Tamanho após transformada: (1, 125, 1)
Tamanho após agrupamento: (1, 125, 1)
Tamanho após transformada: (1, 125, 1)
Tamanho após agrupamento: (1, 125, 1)
Tamanho após transformada: (1, 31, 1)
Tamanho após agrupamento: (1, 31, 1)
Tamanho após transformada: (1, 31, 1)
Tamanho após agrupamento: (1, 31, 1)
Tamanho após transformada: (1, 31, 1)
Tamanho após agrupamento: (1, 31, 1)
Tamanho após transformada: (1, 31, 1)
Tamanho após agrupamento: (1, 31, 1)
Tamanho após transformada: (1, 31, 1)
Tamanho após agrupamento: (1, 31, 1)
Tamanho após transformada: (1, 31, 1)
Tamanho após agrupamento: (1, 31, 1)
Tamanho após transformada: (1, 31, 1)
Tamanho após agrupamento: (1, 31, 1)
Tamanho após transformada: (1, 31, 1)
Tamanho após agrupamento: (1, 31, 1)
Tamanho após tr

#### Applying the DFT and pooling

In [42]:
groupeddft = []

for block in range(len(signals15_blks)):
    dftdata = dataDFT(signals15_blks[block])
    print('Tamanho após transformada:', dftdata.shape)
    datapooldft = datapooling(dftdata, axis=1, width=1)
    print('Tamanho após agrupamento:', datapooldft.shape)
    groupeddft.append(datapooldft)
groupeddatadft = jnp.concatenate(groupeddft, axis=1)
print('Tamanho do sinal com blocos concatenados:', groupeddatadft.shape)
norm_groupeddatadft = NormalizeData(groupeddatadft)

Tamanho após transformada: (1, 500, 1)
Tamanho após agrupamento: (1, 500, 1)
Tamanho após transformada: (1, 125, 1)
Tamanho após agrupamento: (1, 125, 1)
Tamanho após transformada: (1, 125, 1)
Tamanho após agrupamento: (1, 125, 1)
Tamanho após transformada: (1, 125, 1)
Tamanho após agrupamento: (1, 125, 1)
Tamanho após transformada: (1, 125, 1)
Tamanho após agrupamento: (1, 125, 1)
Tamanho após transformada: (1, 31, 1)
Tamanho após agrupamento: (1, 31, 1)
Tamanho após transformada: (1, 31, 1)
Tamanho após agrupamento: (1, 31, 1)
Tamanho após transformada: (1, 31, 1)
Tamanho após agrupamento: (1, 31, 1)
Tamanho após transformada: (1, 31, 1)
Tamanho após agrupamento: (1, 31, 1)
Tamanho após transformada: (1, 31, 1)
Tamanho após agrupamento: (1, 31, 1)
Tamanho após transformada: (1, 31, 1)
Tamanho após agrupamento: (1, 31, 1)
Tamanho após transformada: (1, 31, 1)
Tamanho após agrupamento: (1, 31, 1)
Tamanho após transformada: (1, 31, 1)
Tamanho após agrupamento: (1, 31, 1)
Tamanho após tr

#### Plotting the final coefficients

In [66]:
data = [norm_groupeddata, norm_groupeddatadft]

# Define the colors for each level
colors = [['skyblue', 'steelblue', 'navy'], 
          ['lightgreen', 'limegreen', 'darkgreen']]  # Add more colors if needed

# Create a figure with subplots
fig = make_subplots(rows=len(data), cols=1, shared_xaxes=True, vertical_spacing=0.05,
                    subplot_titles=("Fourier Transform", "Hartley Transform"))

# Add traces for each dataset
for i in range(len(data)):
    # Slice the data into three levels for each dataset
    level1 = data[i][:, :500, :]
    level2 = data[i][:, 500:500+(4*125), :]
    level3 = data[i][:, 500+(4*125):, :]

    # Create traces for each level for the current dataset
    trace1 = go.Scatter(x=np.arange(len(level1[0])), y=level1[0,:,0], mode='lines', name='Level 1', line=dict(color=colors[i][0], width=2))
    trace2 = go.Scatter(x=np.arange(len(level1[0]), len(level1[0])+len(level2[0])), y=level2[0,:,0], mode='lines', name='Level 2', line=dict(color=colors[i][1], width=2))
    trace3 = go.Scatter(x=np.arange(len(level1[0])+len(level2[0]), len(data[i][0])), y=level3[0,:,0], mode='lines', name='Level 3', line=dict(color=colors[i][2], width=2))

    # Add traces to the subplot
    fig.add_trace(trace1, row=i+1, col=1)
    fig.add_trace(trace2, row=i+1, col=1)
    fig.add_trace(trace3, row=i+1, col=1)

# Update layout
#fig.update_layout( title_text="Data Comparison", showlegend=True)

# Show the figure
# Update the layout and display the figure
fig.update_layout(height=400*len(data), title='Coefficients for the nn', template='plotly_white')
fig.show()